In [11]:
import json
import glob
import numpy as np
import pandas as pd
import csv
import string
import re
import math
import sklearn as sk
import nltk
import heapq
import matplotlib.pyplot as plt
import pickle
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk import pos_tag
from sklearn.svm import SVC
from sklearn.model_selection import learning_curve
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

In [12]:
train_files = glob.glob("../input/sample1/SmartLawDataset-main/json/*.json")

In [13]:
mergejson=pd.DataFrame()
for file in train_files:
    with open(file) as f1: 
        data1 = json.load(f1)
    df1=pd.DataFrame([data1])
    mergejson=pd.concat([mergejson,df1])
mg1=mergejson.drop(['header','background','order','footer','annotationProcessingStage','annotationProcessingStageAnnotations','processedText'], axis=1)

In [14]:
header=['ARGUMENT','TYPE']
filename = 'final_argument_by.csv'
data1=[]
with open(filename, 'w') as file:
    csvwriter = csv.writer(file) # 2. create a csvwriter object
    csvwriter.writerow(header) # 4. write the header
    for j in mg1['arguments']:
        for k in j:           
            for p in k['argumentSentences']:                
                data=[]
                if(p['argumentSentenceType']=='NA'):
                    continue
                data.append(p['text'][:])
                data.append(p['argumentSentenceType'])
                data1.append(data)
    csvwriter.writerows(data1) # 5. write the rest of the data
dataset = pd.read_csv('final_argument_by.csv')
df = pd.DataFrame(dataset)

In [15]:
s =  set(string.punctuation)
s.add('\xad')
for index, row in df.iterrows():
    row['ARGUMENT']=row['ARGUMENT'].lower()
    for x in row['ARGUMENT']:
        if x in s or re.search(r'-?\d+', x): 
            row['ARGUMENT']=row['ARGUMENT'].replace(x,"").strip()
#Now we have proper dataset after removing unwanted punctuations etc
arr=[]
arr1=[]
count=0
c1=0
for index, row in df.iterrows():
    arr.append(row['ARGUMENT'])
    arr1.append(row['TYPE'])

df1=pd.DataFrame()
df1["argument"] = arr
df1["by"] = arr1

In [16]:
#list of all sentences
final_list=[]
for item in df1['argument']:
    final_list.append(item)

In [17]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(final_list)
feature_names = vectorizer.get_feature_names_out()
dense = vectors.todense()
denselist = dense.tolist()
columns=feature_names
header=columns
df3 = pd.DataFrame(denselist, columns=feature_names) 
df3['TYpe']=df1['by']

In [18]:
y=df3.TYpe
features=header
x=df3[features]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)

In [20]:
#KNN model
knn = KNeighborsClassifier(n_neighbors= 5) # k=3
knn.fit(X_train,y_train)
prediction = knn.predict(X_test)

In [21]:
#print("{} nın score: {}".format(3,knn.score(X_test,y_test)))

In [22]:
pickle.dump(knn, open('model_knn.pkl', 'wb'))

In [23]:
pickled_model = pickle.load(open('model_knn.pkl', 'rb'))

In [24]:
s_vect=pickle.dumps(vectorizer)
final_model_vect=pickle.loads(s_vect)
model_file_vect = "model_vect.pickle"
with open(model_file_vect,'wb') as f:
    pickle.dump(final_model_vect, f)